In [91]:
import os
import sys
import glob
import re
from pathlib import Path

import numpy as np
import pandas as pd

from sklearn.preprocessing import MinMaxScaler

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.style
import matplotlib as mpl
mpl.style.use('classic')

import pydicom as dcm
import pylab

from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual, Layout
import ipywidgets as widgets
layout=Layout(width='90%')

# # Root directory of the project
# ROOT_DIR = os.path.abspath("../Mask_RCNN/")

# # Import Mask RCNN
# sys.path.append(ROOT_DIR)  # To find local version of the library
# from mrcnn import utils
# from mrcnn import visualize
# from mrcnn.visualize import display_images
# import mrcnn.model as modellib
# from mrcnn.model import log

# See https://stackoverflow.com/questions/14722540/smoothing-between-pixels-of-imagesc-imshow-in-matlab-like-the-matplotlib-imshow
mpl.rcParams['image.interpolation'] = 'none' # default 'bilinear'


In [92]:
home = str(Path.home())
data_dir = f"{home}/data/aneurysm/"
mask_dir = f"{home}/data/aneurysm/512Auto.ours.masks/"
stl_dir = f"{home}/data/aneurysm/stls.ours/"
dcm_dir = f"{home}/data/aneurysm/512Auto.ours/"
pngs_dir = f"{home}/data/aneurysm/512Auto.pngs/"
masks_pngs_dir = f"{home}/data/aneurysm/512Auto.masks.pngs/"

dirs = [data_dir,mask_dir,stl_dir,dcm_dir,pngs_dir,masks_pngs_dir]

In [93]:
# for d in dirs:
#      !mkdir -p {d}
!tree --filelimit=5 {data_dir}
!pwd

/home/amer/data/aneurysm/ [7 entries exceeds filelimit, not opening dir]

0 directories, 0 files
/home/amer/Code/ML_for_Medical_Image_Processing


In [94]:
reg_complete_filename = re.compile("[^/]*$") # Match a file name in a path
reg_filename = re.compile("^([^.]+)") # Match a file name in a path

def get_filename(paths,num):
    complete_filename = re.search(reg_complete_filename, paths[num]).group(0)
    return re.search(reg_filename, complete_filename).group(0)
    
def get_dicoms_paths(source_dir):
    return sorted(glob.glob(f"{source_dir}*.dcm"))

In [95]:
def get_pngs(dicoms_paths,dcm_num):
    ds = dcm.read_file(dicoms_paths[dcm_num]) # get the dicom file #dcm_num
    return ds.pixel_array

# Shows one image
def browse_dicom(source_dir,dicom_num):
    dicoms_paths = get_dicoms_paths(source_dir)
    pngs = get_pngs(dicoms_paths,dicom_num)
    last = len(pngs)-1
    print(dicoms_paths[dicom_num])
    def show_png(num):
        pylab.imshow(pngs[num],cmap=pylab.cm.bone)
    interact(show_png, num=widgets.IntSlider(min=0,max=last,step=1,layout=layout));

# Shows original and mask side by side
def browse_dicom_and_mask(original_dir,mask_dir,dicom_num):
    original_paths = get_dicoms_paths(original_dir)
    mask_paths = get_dicoms_paths(mask_dir)
    
    original_pngs = get_pngs(original_paths,dicom_num)
    mask_pngs = get_pngs(mask_paths,dicom_num)
    
    last = len(original_pngs)-1
    print(original_paths[dicom_num])
    print(mask_paths[dicom_num])
    def show_png(num):
        f = plt.figure(figsize=(20,10))
        f.add_subplot(1,2, 1)
        pylab.imshow(original_pngs[num],cmap=pylab.cm.bone)
        f.add_subplot(1,2, 2)
        pylab.imshow(mask_pngs[num],cmap=pylab.cm.bone)
        plt.show(block=True)
    interact(show_png, num=widgets.IntSlider(min=0,max=last,step=1,layout=layout))

In [96]:
browse_dicom_and_mask(dcm_dir,mask_dir,1)

/home/amer/data/aneurysm/512Auto.ours/A013.dcm
/home/amer/data/aneurysm/512Auto.ours.masks/A013.mask.dcm


interactive(children=(IntSlider(value=0, description='num', layout=Layout(width='90%'), max=438), Output()), _…

In [97]:
#browse_dicom(dcm_dir,1) # Show me the MRI Dicom file #2
#browse_dicom(mask_dir,1) # Show me the Mask Dicom file #2

In [98]:
def save_dicom_to_pngs(source_dir,target_dir,dicom_num):
    paths = get_dicoms_paths(source_dir)
    dicom_file_name = get_filename(paths,dicom_num)
    pngs = get_pngs(paths,dicom_num)
    slice_count = len(pngs)-1
    for i in range(dicom_num,slice_count):
        pylab.imsave(f"{target_dir}{dicom_file_name}_{i}",pngs[i],cmap=pylab.cm.bone)
    return slice_count+1

In [99]:
def breakup_dicom(source_dir,target_dir): # Dicom to pngs
    dicom_paths = get_dicoms_paths(source_dir) # Get dicoms files
    file_count = len(dicom_paths) # Count dicom files
    for i in range(0,file_count):
        print(f"Breaking up {get_filename(dicom_paths,i)} into pngs...")
        slice_num = save_dicom_to_pngs(source_dir,target_dir,i)
        print(f"Finished {get_filename(dicom_paths,i)}! {slice_num} slices were saved.\n")
    print(f"PNGs are in {pngs_dir}")

In [100]:
#breakup_dicom(dcm_dir,pngs_dir)

In [101]:
#breakup_dicom(mask_dir,masks_pngs_dir)

In [102]:
# return the dicom file as a numpy array
# @num is the index of the file in "dcm_dir" directory
def dicom_as_tensor(index):
    return get_pngs(get_dicoms_paths(dcm_dir),index)

In [103]:
def show_scaled_dicom(source_dir,dcm_index):
    dicoms_paths = get_dicoms_paths(source_dir)
    dcm = get_pngs(dicoms_paths,dcm_index)
    last = last = len(dcm)-1
    print(last)
    def show_img(num):
        _slice = dcm[num]
        scaler = MinMaxScaler(feature_range=(0, 1))
        scaler.fit(_slice)
        scaled = scaler.transform(_slice)
        plt.figure(figsize=(8,8))
        pylab.imshow(scaled,cmap=pylab.cm.bone)
    interact(show_img, num=widgets.IntSlider(min=0,max=last,step=1,layout=layout));

In [110]:
show_scaled_dicom(dcm_dir,3)

438


interactive(children=(IntSlider(value=0, description='num', layout=Layout(width='90%'), max=438), Output()), _…

In [125]:
tensor = dicom_as_tensor(3)[0]
# print(np.shape(tensor))
# w = 200 # window1
# m = 1 # stried
print(np.shape(tensor))
print(tensor[m:m+w,m:m+w])
# plt.figure(figsize=(8,8))
# pylab.imshow(tensor,cmap=pylab.cm.bone)

(512, 512)
[[-373 -446 -278 ... -538 -407 -438]
 [-523 -401 -404 ... -398 -433 -370]
 [-579 -497 -444 ... -303 -379 -369]
 ...
 [-492 -560 -523 ... -618 -599 -344]
 [-500 -398 -223 ... -574 -486 -128]
 [-346 -272 -270 ... -328 -253  -51]]


In [188]:
# Sliding window
def tensor_window(tensor):
    height = np.shape(tensor)[0]
    width = np.shape(tensor)[1]
    def move_window(x,y,w):
        print(np.shape(tensor))
        print(f'[x-w:x,y-w:y]: [{max([x-w,0])}:{x},{max([y-w,0])}:{y}]')
        wx = max([x-w,0])
        wy = max([y-w,0])
        print(f'Window= {w}x{w}')
        t = tensor[wx:x,wy:y]
        pylab.imshow(t,cmap=pylab.cm.bone)
    s = 1 # stried
    interact(move_window,
             x=widgets.IntSlider(min=10,max=height,step=s,layout=layout),
             y=widgets.IntSlider(min=10,max=width,step=s,layout=layout),
             w=widgets.IntSlider(min=20,max=512,step=1,layout=layout),)
tensor_window(tensor)

interactive(children=(IntSlider(value=10, description='x', layout=Layout(width='90%'), max=512, min=10), IntSl…

In [17]:
# فكرة جمع طبقات الصور لزادة احتمال